<a href="https://colab.research.google.com/github/pawntest/vla_play/blob/main/pi_vla_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared
launch_ssh_cloudflared(password="<てきとうなパスワードを記入>")


Mounted at /content/gdrive


In [ ]:
! curl -LsSf https://astral.sh/uv/install.sh | sh
!export PATH="$HOME/.local/bin:$PATH"
! source $HOME/.local/bin/env
! git clone https://github.com/Physical-Intelligence/openpi
%cd openpi
!pwd
!GIT_LFS_SKIP_SMUDGE=1 uv sync

Streaming output truncated to the last 5000 lines.
jax-cuda12-pjrt ------------------------------ 86.37 MiB/98.35 MiB
nvidia-cusolver-cu12 ------------------------------ 85.80 MiB/122.01 MiB
nvidia-cusparselt-cu12 ------------------------------ 85.11 MiB/143.11 MiB
nvidia-nccl-cu12 ------------------------------ 85.45 MiB/179.91 MiB
nvidia-cusparse-cu12 ------------------------------ 84.76 MiB/197.84 MiB
nvidia-cufft-cu12 ------------------------------ 84.87 MiB/201.66 MiB
triton     ------------------------------ 85.96 MiB/241.43 MiB
nvidia-cublas-cu12 ------------------------------ 85.33 MiB/346.60 MiB
nvidia-cudnn-cu12 ------------------------------ 84.93 MiB/633.96 MiB
⠼ Preparing packages... (230/241)
jaxlib     ------------------------------ 87.23 MiB/97.25 MiB
jax-cuda12-pjrt ------------------------------ 86.66 MiB/98.35 MiB
nvidia-cusolver-cu12 ------------------------------ 86.20 MiB/122.01 MiB
nvidia-cusparselt-cu12 ------------------------------ 85.31 MiB/143.11 MiB
nvidia-

In [ ]:
# Create virtual environment
!uv venv --python 3.10 examples/aloha_sim/.venv
!source examples/aloha_sim/.venv/bin/activate
!uv pip sync examples/aloha_sim/requirements.txt
!uv pip install -e packages/openpi-client

Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: examples/aloha_sim/.venv
Activate with: source examples/aloha_sim/.venv/bin/activate
Using Python 3.11.11 environment at: /usr
Resolved 47 packages in 486ms
⠙ Preparing packages... (0/8)
⠙ Preparing packages... (0/8)
urllib3    ------------------------------     0 B/123.38 KiB
⠙ Preparing packages... (0/8)
urllib3    ------------------------------ 14.88 KiB/123.38 KiB
⠙ Preparing packages... (0/8)
urllib3    ------------------------------ 30.88 KiB/123.38 KiB
⠙ Preparing packages... (0/8)
urllib3    ------------------------------ 46.88 KiB/123.38 KiB
⠙ Preparing packages... (0/8)
urllib3    ------------------------------ 62.88 KiB/123.38 KiB
⠙ Preparing packages... (0/8)
urllib3    ------------------------------ 62.88 KiB/123.38 KiB
psutil     ------------------------------     0 B/280.52 KiB
⠙ Preparing packages... (0/8)
tyro       ------------------------------     0 B/109.51 KiB
urllib3    ---

In [ ]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()


In [ ]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)